In [27]:
import sys
sys.path.append('../')

In [28]:
from brix import Indicator, Handler, Grid_maker
import US_State
import datetime
import geopandas as gpd
import json
import requests

# Initialise the table

In [29]:
table_name='epa'
state_fips=6
geom_type='block'
lat,lon = 37.47, -122.15634055
cell_size=55
nrows=50
ncols=50
rotation=30

In [30]:
G = Grid_maker(table_name, lat, lon, 
               cell_size=cell_size, nrows=nrows, ncols=ncols, rotation=rotation)

https://cityscope-api.smartaraucania.org/api/tables/list/ Attempt: 0
Table epaversion already exists
Calculating initial coordinates of each cell
Defining properties and headers


c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\grid_maker.py:82: UserWarning: Table epaversion already exists
  warn(f'Table {self.table_name} already exists')
c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\grid_maker.py:102: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  top_left_xy=pyproj.transform(self.wgs, self.projection,
c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\grid_maker.py:104: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  top_right_xy=pyproj.transform(self.wgs, self.projection,
c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\grid_maker.py:119: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lat_grid, lon_grid=py

In [31]:
geogrid=G.get_grid_geojson()

Generating grid geojson


### Define the cell types

In [32]:
types=json.load(open('../data/default_types.json'))
geogrid['properties']['types']=types

### Initialise the grid cells all to None type

In [33]:
for i in range(len(geogrid['features'])):
    geogrid['features'][i]['properties']['name']='None'
    geogrid['features'][i]['properties']['color']=types['None']['color']

### Set some cells to be interactive based on overlap with a boundary

In [34]:
interactive_area=gpd.read_file(
"""
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.1340012550354,
              37.48372979560496
            ],
            [
              -122.13342189788817,
              37.482384632757686
            ],
            [
              -122.13419437408449,
              37.47972829068993
            ],
            [
              -122.13535308837889,
              37.47724214141305
            ],
            [
              -122.13432312011719,
              37.47727619881123
            ],
            [
              -122.13428020477295,
              37.474108794382126
            ],
            [
              -122.13052511215209,
              37.47436423519954
            ],
            [
              -122.13061094284058,
              37.475692513376735
            ],
            [
              -122.12966680526732,
              37.47575211504922
            ],
            [
              -122.12964534759521,
              37.47693562413104
            ],
            [
              -122.1289587020874,
              37.47696968166888
            ],
            [
              -122.13181257247923,
              37.47937069896909
            ],
            [
              -122.129967212677,
              37.48372979560496
            ],
            [
              -122.1340012550354,
              37.48372979560496
            ]
          ]
        ]
      }
    }
  ]
}
""")

In [35]:
overlap_ids=gpd.overlay(interactive_area,
                        gpd.read_file(json.dumps(geogrid)), 
                        'intersection')['id'].values

In [36]:
for i in range(len(geogrid['features'])):
    if i in overlap_ids:
        geogrid['features'][i]['properties']['interactive']=True
    else:
        geogrid['features'][i]['properties']['interactive']=False
        geogrid['features'][i]['properties']['color']=[50,50,50,50]

### Send the grid data to CityIO

In [37]:
headers = {'Content-Type': 'application/json'}
url='https://cityscope-api.smartaraucania.org/api/table/{}'.format(table_name)
r = requests.post(url+'/GEOGRID', data = json.dumps(geogrid), headers=headers)

For the change to be reflected on the front-end, we need to update the current grid state based on the changes to the GEOGRID

In [38]:
G.reset_geogrid_data()

https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: wTFjHvG8e6CSSBeuuwjArjh2fcP5GUNMpwkeTgRWdif


The front-end is now ready but there are no analysis modules running yet so nothing happens when we interact with the grid